In [1]:
import pandas as pd
import numpy as np
from numpy import vectorize
import math
songs=pd.read_csv("msdchallenge/kaggle_songs.txt", sep=" ", header=None)
users=pd.read_csv("msdchallenge/kaggle_users.txt", header=None)
#song_to_track=pd.read_csv("msdchallenge/taste_profile_song_to_tracks.txt",sep="\t",header=None)

valHid = pd.read_csv("EvalDataYear1MSDWebsite/year1_valid_triplets_visible.txt", sep="\t", header=None)

valVis = pd.read_csv("EvalDataYear1MSDWebsite/year1_valid_triplets_hidden.txt", sep="\t", header=None)

testVis = pd.read_csv("EvalDataYear1MSDWebsite/year1_test_triplets_visible.txt", sep="\t", header=None)

testHid = pd.read_csv("EvalDataYear1MSDWebsite/year1_test_triplets_hidden.txt", sep="\t", header=None)


songs.columns=["song_id","song_index"]
users.columns=["user_id"]
valHid.columns = ["user_id","song_id","counts"]
valVis.columns = ["user_id","song_id","counts"]

testVis.columns=["user_id","song_id","counts"]
testHid.columns=["user_id","song_id","counts"]
print( list( valHid.user_id.unique() )==list( valVis.user_id.unique() ) ) #users same for val and test
print( list( testHid.user_id.unique() )==list( testVis.user_id.unique() ) )

True
True


In [2]:


testVis = testVis.head(10000) 
testHid = testHid.head(10000) #to accelerate following for loop 
valVis = valVis.head(2000)
valHid = valHid.head(2000)

train = pd.concat([testVis,testHid, valVis])

data = pd.concat([train, valHid, valVis])
print( data.user_id.nunique() )
print( data.song_id.nunique() )
print( valHid.user_id.nunique() )
print( valVis.user_id.nunique() )
print( testHid.user_id.nunique() )
print( testVis.user_id.nunique() )


subsongs = pd.Series( data.song_id.unique() )#.sample(frac=1/50, random_state=1)
subusers=pd.Series( data.user_id.unique() )#.sample(frac=0.2, random_state=1)
#to decrease matrix size and a random sampling, but this sampling can make matrix too sparse

895
15301
163
158
704
732


In [3]:

matrix = pd.DataFrame( np.zeros( (subusers.__len__(), subsongs.__len__()) ) )
matrix.columns=list(subsongs)
matrix.index=list(subusers)
# setsongs=set(subsongs)
# setusers=set(subusers)


for i in range(train.__len__()): #iloc: position based, loc: index(or column name) based
    #print(i)
    r=train.iloc[i].loc['user_id']
    c=train.iloc[i].loc['song_id']
#     if r in setusers and c in setsongs:
    matrix.loc[r].loc[c]=train.iloc[i].loc['counts']
    #log scale conversion create extremly large predictions...should not take log/Users/yzh/Desktop/DataM/msdchallenge/cosRec.py

In [4]:

def getusermat(matrix, subsongs, subusers): #user user based
    vfunc = vectorize( lambda x: 0 if x==0 else 1/( math.sqrt(x) ) )
    R = matrix.values # userItem matrix R
    Rt = R.transpose()
    P = np.diag( np.sum(R, axis=1) )   #row sum as diag

    P2 = vfunc(P)

    Rt = R.transpose()
    res = P2 @ R @ Rt @ P2 @ R
    res = pd.DataFrame( res )
    #res.columns=list(subsongs)
    #res.index=list(subusers)
    return res

usermat = getusermat(matrix, subsongs, subusers)


In [6]:
def getusermat2(matrix, subsongs, subusers): #item item based
    vfunc = vectorize( lambda x: 0 if x==0 else 1/( math.sqrt(x) ) )
    R = matrix.values # userItem matrix R
    Rt = R.transpose()
    Q = np.diag( np.sum(R, axis=0) )   #col sum as diag

    Q2 = vfunc(Q)

    Rt = R.transpose()
    res = R @ Q2 @ Rt @ R @ Q2
    res = pd.DataFrame( res )
    #res.columns=list(subsongs)
    #res.index=list(subusers)
    return res

usermat2 = getusermat2(matrix, subsongs, subusers)
usermat3=(usermat+usermat2)/2 #blend model
usermat3.columns=list(subsongs)
usermat3.index=list(subusers)

In [7]:
pred=[]
for i in range( valHid.__len__() ):
    r=valHid.iloc[i].loc['user_id']
    c=valHid.iloc[i].loc['song_id']
    pred.append( usermat3.loc[r].loc[c])
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = math.sqrt( mean_squared_error(pred, valHid.counts) ) #rms 50
valHid['pred'] = pred

In [12]:
rms

35.561320401861806

In [8]:
from sklearn.metrics import average_precision_score
from scipy.stats import spearmanr
def getmap(df):#get mean average precision
    d={uid:[[], []] for uid in df['user_id']} #[[count], [pred]]
    for i in range( df.__len__() ):
        series = df.iloc[i]
        userid=series.loc['user_id']
        d[userid][0].append( float(series.loc['counts']) )
        d[userid][1].append( float(series.loc['pred']) )
    isum=0
    count=0
#     print(d)
    for key in d:
        #print(d[key][0], d[key][1])
        if not math.isnan( spearmanr( d[key][0], d[key][1] )[0]):# zero varianve leads to nan
          isum+=spearmanr( d[key][0], d[key][1] )[0]
          count+=1
#         else:
#           print(d[key][0], d[key][1])
            
    return count, isum/count, d.__len__()
getmap(valHid)

/Users/yzh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/yzh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/yzh/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/yzh/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/yzh/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


(89, 0.04362601946305621, 163)

In [9]:
trainPred=[]
for i in range( train.__len__() ):
    r=train.iloc[i].loc['user_id']
    c=train.iloc[i].loc['song_id']
    trainPred.append( usermat3.loc[r].loc[c])
from sklearn.metrics import mean_squared_error
from math import sqrt

trainRms = math.sqrt( mean_squared_error(trainPred, train.counts) ) #train rms 750?, rms not a good metric
train['pred'] = trainPred 
getmap(train)# at least training data's average spearman R does not suck like rms

/Users/yzh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/yzh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/yzh/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/yzh/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/yzh/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


(825, 0.7522746980440519, 890)

In [13]:
trainRms

1170.435166873333

In [10]:
train.shape

(22000, 4)

In [11]:
train

,user_id,song_id,counts,pred
0,00007a02388c208ea7176479f6ae06f8224355b3,SOAITVD12A6D4F824B,3,22.945019
1,00007a02388c208ea7176479f6ae06f8224355b3,SONZGLW12A6D4FBBC1,1,7.968498
2,00007a02388c208ea7176479f6ae06f8224355b3,SOXNWYP12A6D4FBDC4,1,12.617749
3,00007a02388c208ea7176479f6ae06f8224355b3,SOSPTOE12A6D4F938C,1,10.158883
4,00007a02388c208ea7176479f6ae06f8224355b3,SOXMOQF12A6701FB8B,1,7.965780
5,00007a02388c208ea7176479f6ae06f8224355b3,SOSMMEW12A8C134F96,2,18.347969
6,00007a02388c208ea7176479f6ae06f8224355b3,SONGABP12A8C1442D4,1,12.617749
7,00007a02388c208ea7176479f6ae06f8224355b3,SOJTNQQ12AF72A246E,2,18.347969
8,00007a02388c208ea7176479f6ae06f8224355b3,SOTNDOM12AF729CADB,1,12.617749
9,00014a76ed063e1a749171a253bca9d9a0ff1782,SORHASY12A6D4F8664,1,110.117939
